
# 1 - Imports and defining functions

In [ ]:
import numpy as np

import meshplot as mp
import matplotlib.pyplot as plt

from pyFM.functional import FunctionalMapping
from project.evaluate import EvaluateModel # evaluation class
from pyFM.functional import FunctionalMapping # functional mapping class
from project.cfunctional import CoupledFunctionalMapping # coupled functional mapping class
from project.datasets import ShrecPartialDataset as SPD # Shrec dataset class
from project.datasets import FaustRep as FR # Faust dataset class
from project.datasets import Tosca as T # Tosca dataset class

def plot_mesh(myMesh,cmap=None):
    mp.plot(myMesh.vertlist, myMesh.facelist,c=cmap)
    
def double_plot(myMesh1,myMesh2,cmap1=None,cmap2=None):
    d = mp.subplot(myMesh1.vertlist, myMesh1.facelist, c=cmap1, s=[2, 2, 0])
    mp.subplot(myMesh2.vertlist, myMesh2.facelist, c=cmap2, s=[2, 2, 1], data=d)

def visu(vertices):
    min_coord,max_coord = np.min(vertices,axis=0,keepdims=True),np.max(vertices,axis=0,keepdims=True)
    cmap = (vertices-min_coord)/(max_coord-min_coord)
    return cmap

# 2- Evaluating Results

Definition of paths, **adjust to your local paths**

In [ ]:
data_path_cp2p = "../../data/cp2p_dev"
data_path_faust = "../../data/MPI-FAUST_dev"
data_path_tosca = "../../data/SGP_dataset_off"

### Functional mapping on FAUST representation data

- preprocess_params arguments are given to model.preprocess()
- fit_params arguments are given to model.fit()
- data_params arguments are given to the dataclass initialization

In [ ]:
evaluation = EvaluateModel(
    FunctionalMapping, FR, data_path_faust, refine=False, 
    preprocess_params={
        'n_ev': (35,35),  # Number of eigenvalues on source and Target
        # 'landmarks': np.loadtxt('data/landmarks.txt',dtype=int)[:5],  # loading 5 landmarks
        'subsample_step': 5,  # In order not to use too many descriptors
        'descr_type': 'WKS',  # WKS or HKS
    },
    fit_params={
        'w_descr': 1e0,
        'w_lap': 1e-2,
        'w_dcomm': 1e-1,
        'w_orient': 0
    },
    # data_params={
    #     'name': 'cuts',
    #     'selected': True
    # },
    verbose=False)

Call eval() function to evaluate the given dataset on the given model

In [ ]:
evaluation.eval()

### Coupled Functional Mapping on Tosca Data 

In [ ]:
evaluation_c = EvaluateModel(
    CoupledFunctionalMapping, T, data_path_tosca, refine=False, 
    preprocess_params={
        'n_ev': (35,35),  # Number of eigenvalues on source and Target
        # 'landmarks': np.loadtxt('data/landmarks.txt',dtype=int)[:5],  # loading 5 landmarks
        'subsample_step': 5,  # In order not to use too many descriptors
        'descr_type': 'WKS',  # WKS or HKS
    },
    fit_params={
        'mu_coup': 1e-1,
        'mu_mask': 1e-2,
        'mu_des': 1e-1,
    },
    data_params={
        'name': 'cuts',
        'selected': True,
        'use_adj': True
    },
    verbose=False)

In [ ]:
evaluation_c.eval()

### Functional Mapping on Tosca Data

In [ ]:
evaluation_fm = EvaluateModel(
    FunctionalMapping, T, data_path_tosca, refine=False, 
    preprocess_params={
        'n_ev': (35,35),  # Number of eigenvalues on source and Target
        # 'landmarks': np.loadtxt('data/landmarks.txt',dtype=int)[:5],  # loading 5 landmarks
        'subsample_step': 5,  # In order not to use too many descriptors
        'descr_type': 'WKS',  # WKS or HKS
    },
    fit_params={
        'w_descr': 1e0,
        'w_lap': 1e-2,
        'w_dcomm': 1e-1,
        'w_orient': 0
    },
    data_params={
        'name': 'cuts',
        'selected': True,
        'use_adj': True
    },
    verbose=False)

In [ ]:
evaluation_fm.eval()

### Load results

**adjust paths to your local paths**

In [ ]:
def load_res(path):
    with open(path, "r") as f:
        return [float(x) for x in f.read().split(", ")]


FM_res = load_res(f'data/eval/distances_FunctionalMapping_Tosca.txt')
CFM_res = load_res(f'data/eval/distances_CoupledFunctionalMapping_Tosca.txt')

# FM_res = load_res(f'data/eval/distances_FunctionalMapping_FaustRep.txt')
# CFM_res = load_res(f'data/eval/distances_CoupledFunctionalMapping_FaustRep.txt')

### Plot results

In [ ]:
def plot_res(x, y):
    x = np.sort(x)
    f_x = np.array(range(len(x)))/float(len(x))
    y = np.sort(y)
    f_y = np.array(range(len(y)))/float(len(y))

    plt.plot(x, f_x, label='FM')
    plt.plot(y, f_y, label='CFM')
#     plt.title('Geod. distance distribution (FAUST representations)')
    plt.title('Geod. distance distribution (Tosca)')
    plt.xlabel('Geodesic error')
    plt.ylabel('% Correspondences')
    plt.xlim([0.0, 1.5])
    plt.ylim([0.0, 1.0])
    plt.legend()

#     plt.savefig(f"data/eval/FM_v_CFM_Faust.png", bbox_inches='tight', dpi=200)
    plt.savefig(f"data/eval/FM_v_CFM_Tosca.png", bbox_inches='tight', dpi=200)

In [ ]:
plot_res(FM_res, CFM_res)